In [1]:
import os
import pandas as pd
import janitor
import numpy as np
pd.set_option('display.max_columns', 100)

DATAPATH = "../../adult-data"        

In [2]:
# (fold cell) Load web browsing data
usecols_web_desktop = ["caseid", "category", "private_domain", "visit_duration", "visit_time_local"]
usecols = ["caseid", "category", "private_domain", "page_duration", "session_start_time"]

df = (pd.concat([
        # Get web_mobile
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_mobile_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ), 
        # Get web_desktop
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_desktop_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ),
        # Get web
        (pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_2022-06-01_2022-06-30.csv"),
            usecols=usecols,
            low_memory=False,
        )
         # Renaming columns to be consistent w/ web_mobile & web_desktop
         .rename_column("session_start_time", "visit_time_local")
         .rename_column("page_duration", "visit_duration")
        )
        ])
      .assign(caseid=lambda df: df.caseid.astype(object))      
     )
print("df of web browses")
print(f"{len(df)=:,}")
df.head()

df of web browses
len(df)=5,090,441


,caseid,private_domain,category,visit_time_local,visit_duration
0,210362181,liveineverett.com,NaN,2022-05-31 17:19:01,50
1,210362181,wa.gov,"Education, Government",2022-05-31 19:28:41,48
2,210362181,yahoo.com,Economy and Finance,2022-06-01 03:16:33,1
3,210362181,collider.com,Entertainment,2022-06-01 03:29:15,60
4,210362181,yahoo.com,"Education, News and Media",2022-06-01 03:42:43,83


In [18]:
df_caseid = (df
             .rename_column("visit_duration", "seconds")
             .groupby("caseid")["seconds"].sum().reset_index()
             .sort_values("seconds", ignore_index=True)
             .assign(
                 mins=lambda df: df.seconds/60,
                 hours=lambda df: df.mins/60,
             )
            )
df_caseid.head(50)

,caseid,seconds,mins,hours
0,286663881,2,0.033333,0.000556
1,206433705,5,0.083333,0.001389
2,301913121,6,0.100000,0.001667
3,8085031,7,0.116667,0.001944
4,80623593,8,0.133333,0.002222
5,286591993,36,0.600000,0.010000
6,307997169,38,0.633333,0.010556
7,240099429,46,0.766667,0.012778
8,8953795,46,0.766667,0.012778
9,320316041,62,1.033333,0.017222


In [21]:
len(df_caseid.query("hours<1"))

49

In [22]:
len(df_caseid.query("hours<2"))

83

In [24]:
len(df_caseid.query("hours<5"))

152